# Download Modules

In [1]:
!pip install fairseq

In [2]:
!pip install transformers sentencepiece datasets

In [3]:
from datasets import load_dataset, arrow_dataset
# from IPython.display import display
# from IPython.html import widgets
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook
from fairseq.models.bart import BARTModel
from transformers import BartForConditionalGeneration, BartTokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
# Device configuration
device = 0
if torch.cuda.is_available():
    device = torch.device('cuda:0') 
    torch.cuda.set_per_process_memory_fraction(fraction = 0.92, device = device)
else:
    device = torch.device('cpu')

print(device)

cuda:0


In [5]:
# CONFIGURATION
weight_path_save = '/kaggle/working/'
data_batch_size = 16
max_seq_len = 160
LANG_TOKEN_MAPPING = {
                    'en': '<en>',
                    'vi': '<vi>' }
# Constants
n_epochs = 50
print_freq = 50
checkpoint_freq = 1000
lr = 1e-4

# Load Tokenizer and Model

In [6]:
# Load pretrained model
bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
bart.eval()  # disable dropout (or leave in train mode to finetune)
# Load tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Test tokenizer and model

In [7]:
token_ids = tokenizer(text = 'This will be translated to Japanese! (hopefully)', 
                      padding='max_length', truncation=True,return_tensors='pt',max_length=max_seq_len)
print(token_ids)
model_out = bart.generate(token_ids['input_ids'])
# decoding the generated encoded ids
print(tokenizer.decode(model_out[0], skip_special_tokens=True))

{'input_ids': tensor([[    0,   713,    40,    28, 16877,     7,  2898,   328,    36,  9547,
         24358,    43,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,  

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


This will be translated to Japanese! (hopefully)


# Prepare Dataset

In [8]:
# Source: https://huggingface.co/datasets/alt
# loading dataset
dataset = load_dataset('alt')
# get training set and validation set
train_dataset = dataset['train']
test_dataset = dataset['test']

  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
test_dataset[0]

{'SNT.URLID': '78103',
 'SNT.URLID.SNTID': '34',
 'url': 'http://en.wikinews.org/wiki/Horse_flu_infects_Australian_thoroughbreds',
 'translation': {'bg': 'সিডনির র\u200d্যান্ডউইক রেসের মাঠের আটটি থরোব্রেড রেসের ঘোড়া্র এক্যুইন ইনফ্লুয়েঞ্জাতে আক্রান্ত হওয়ার খবরটি নিশ্চিত করা হয়েছে।',
  'en': 'It has been confirmed that eight thoroughbred race horses at Randwick Racecourse in Sydney have been infected with equine influenza.',
  'en_tok': 'It has been confirmed that eight thoroughbred race horses at Randwick Racecourse in Sydney have been infected with equine influenza .',
  'fil': 'Kumpirmado na walong magandang lahing pangkarerang kabayo sa Randwick Racecourse sa Sydney ang nagkaroon ng trangkaso.',
  'hi': 'यह पुष्टि की गई है कि सिडनी में रैंडविक रेसकोर्स के आठ घनी नस्ल के घोड़े इक्वाइन इन्फ्लूएंजा से संक्रमित हैं।',
  'id': 'Telah dikonfirmasi bahwa 8 kuda pacu thoroughbred di pacuan kuda Randwick di Sydney telah terinfeksi oleh flu kuda.',
  'ja': 'シドニーのランドウィック競馬場の8頭のサラブレッド競走馬が馬インフルエン

In [10]:
test_dataset[2]['translation']

{'bg': 'এটাই প্রত্যাশিত যে এই অত্যন্ত ক্ষতিকারক ফ্লু-টি র\u200d্যান্ডউইকের আস্তাবলের ৭০০ ঘোড়ার অধিকাংশকেই আক্রমণ করবে।',
 'en': 'It is expected that the virulent flu will affect the majority of the 700 horses stabled at Randwick.',
 'en_tok': 'It is expected that the virulent flu will affect the majority of the 700 horses stabled at Randwick .',
 'fil': 'Inaasahan na ang malubhang sakit ay makakaapekto sa higit 700 na kabayong nasa kuwadra sa Randwick.',
 'hi': 'यह माना गया है कि वीरूलेंट फ़्लू रैंडविक में रखे गए 700 घोड़ों में से अधिकांश को प्रभावित करेगा।',
 'id': 'Diperkirakan bahwa flu mematikan tersebut akan mempengaruhi sebagian besar dari 700 kuda yang dikandangkan di Randwick.',
 'ja': 'その強力なインフルエンザは、ランドウィック競馬場の厩舎にいる700頭の馬の大半に影響すると予想されている。',
 'khm': 'គេប៉ាន់ស្មានថានឹងមានឥទិ្ធពលរីករាលដាលភាគច្រើនដល់សេះចំនួន700ក្បាលដែលស្ថិតនៅរង់វីក។',
 'lo': 'ຄາດວ່າໄຂ້ພິດຈະຕິດເຊື້ອກັບມ້າຈຳນວນຫຼາຍຈາກທັງໝົດ 700 ໂຕ ທີ່ ແຣນວິກ.',
 'ms': 'Dijangkakan selsema virulent itu akan mempengaruhi majoriti 700 

In [11]:
LANG_TOKEN_MAPPING.keys()

dict_keys(['en', 'vi'])

In [12]:
# Difine keys for switch between two languages
forward_map_key = list(LANG_TOKEN_MAPPING.keys())
print(forward_map_key)
backward_map_key = forward_map_key.copy()
backward_map_key.reverse()
print(backward_map_key)

['en', 'vi']
['vi', 'en']


In [13]:
# Define functions for prepocessing data
def _encode_text(input_text: str, tokenizer: BartTokenizer, max_seq_length: int ):
    input_ids = tokenizer(text = input_text, 
                 padding='max_length', 
                 truncation=True,
                 return_tensors='pt',
                 max_length=max_seq_length)
    return input_ids['input_ids']

def _format_translation_data(translations, order_lang_token_map_keys, tokenizer: BartTokenizer, max_seq_length: int):
    """
    order_lang_token_map_keys: list contains keys from 'lang_token_map' with determiant order
    """
    input_lang, target_lang = order_lang_token_map_keys
    # Get the translations for the batch
    input_text = translations[input_lang]
    target_text = translations[target_lang]
    
    if input_text is None or target_text is None:
        return None
    # str to vector
    input_token = _encode_text(input_text, tokenizer, max_seq_length)
    target_token = _encode_text(target_text, tokenizer, max_seq_length)
    return input_token, target_token

def transform_batch(batch: arrow_dataset.Dataset, lang_token_map: dict, tokenizer: BartTokenizer, max_seg_length: int):
    inputs = []
    targets = []
    forward_map_key = list(lang_token_map.keys()) # ENG to VNI
    
    for translation_set in batch['translation']:
        # for from ENG to VNI
        return_data = _format_translation_data(translation_set, forward_map_key,tokenizer, max_seg_length)
        if return_data is None :
            continue
        else:     
            input_token, target_token = return_data
            # for from VNI to ENG
            reversed_input_token, reversed_target_token = target_token, input_token


            # append to master
            inputs.append(input_token)
            targets.append(target_token)

            inputs.append(reversed_input_token)
            targets.append(reversed_target_token)
    
    batch_input_tokens = torch.cat(inputs)
    batch_target_tokens = torch.cat(targets)
    
    return batch_input_tokens, batch_target_tokens

# Define generator
def get_data_generator(dataset: arrow_dataset.Dataset, lang_token_map: dict, tokenizer: BartTokenizer, max_seg_length: int, batch_size=data_batch_size):
    dataset = dataset.shuffle()
    for i in range(0, len(dataset), batch_size):
        raw_batch = dataset[i:i+batch_size]
        yield transform_batch(raw_batch, lang_token_map, tokenizer, max_seg_length)

In [14]:
# Testing data transformation
batch_input_tokens, batch_target_tokens = transform_batch(train_dataset[1:1+5], LANG_TOKEN_MAPPING, tokenizer, max_seq_len)

for order, (input_tk, out_tk) in enumerate(zip(batch_input_tokens, batch_target_tokens)):
    print(order)
    print(tokenizer.decode(input_tk, skip_special_tokens=True))
    print("out ids")
    print(tokenizer.decode(out_tk, skip_special_tokens=True))

0
Andrea Masi opened the scoring in the fourth minute with a try for Italy.
out ids
Andrea Maisi đã mở tỉ số cho Ý ở phút thứ tư với một quả try.
1
Andrea Maisi đã mở tỉ số cho Ý ở phút thứ tư với một quả try.
out ids
Andrea Masi opened the scoring in the fourth minute with a try for Italy.
2
Despite controlling the game for much of the first half, Italy could not score any other tries before the interval but David Bortolussi kicked three penalties to extend their lead.
out ids
Chiếm thế áp đảo trong hầu hết hiệp đầu nhưng Ý đã không thể ghi thêm try nào trước khi nghỉ giữa giờ, tuy nhiên David Bortolussi đã sút ba quả phạt đền kéo dài thế dẫn đầu của họ
3
Chiếm thế áp đảo trong hầu hết hiệp đầu nhưng Ý đã không thể ghi thêm try nào trước khi nghỉ giữa giờ, tuy nhiên David Bortolussi đã sút ba quả phạt đền kéo dài thế dẫn đầu của họ
out ids
Despite controlling the game for much of the first half, Italy could not score any other tries before the interval but David Bortolussi kicked thre

# Train/Finetune BART

In [15]:
# Define optimizer
n_batches = int(np.ceil(len(train_dataset) / data_batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)
# Optimizer
optimizer = AdamW(bart.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, n_warmup_steps, total_steps)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
def eval_model(model, test_dataset_gen):
    eval_losses = []
    for i, (input_batch, label_batch) in enumerate(test_dataset_gen):
        input_batch = input_batch.to(device)
        label_batch = label_batch.to(device)
        model_out = model.forward(input_ids = input_batch,labels = label_batch)
        eval_losses.append(model_out.loss.item())

    return np.mean(eval_losses)

In [ ]:
bart = bart.to(device)
for para in bart.parameters():
    para.requires_grad = True
master_train_losses = []
master_val_losses = []
for epoch_idx in range(n_epochs):
    # Define data generator
    train_gen = get_data_generator(dataset=train_dataset, 
                                   lang_token_map=LANG_TOKEN_MAPPING, 
                                   tokenizer=tokenizer, 
                                   max_seg_length= max_seq_len)
    test_gen = get_data_generator(dataset=test_dataset, 
                                  lang_token_map=LANG_TOKEN_MAPPING, 
                                  tokenizer=tokenizer, 
                                  max_seg_length= max_seq_len)
    print("epoch:", epoch_idx)
    bart.train()
    
    curr_epoch_train_loss = []
    for batch_idx, (input_batch, label_batch) in enumerate(train_gen):
        # clean gradient
        optimizer.zero_grad()
        input_batch = input_batch.to(device)
        label_batch = label_batch.to(device)
        # Forward pass
        model_out = bart.forward(input_ids = input_batch,labels = label_batch)

        # Calculate loss and update weights
        loss = model_out.loss
        curr_epoch_train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Print training update info
    avg_loss=np.mean(curr_epoch_train_loss)
    master_train_losses.append(avg_loss)
    print('Epoch: {} | avg.loss: {:.3f} | lr: {}'.format(epoch_idx, avg_loss, scheduler.get_last_lr()[0]))
    
    bart.eval()
    with torch.no_grad():
        print("Validation")
        test_mean_loss = eval_model(bart, test_gen)
        master_val_losses.append(test_mean_loss)
        current_min_loss_in_total = min(master_val_losses)
        print('Epoch: {} | Validation loss {}'.format(epoch_idx, test_mean_loss))
        
        # save model weigth
        if  test_mean_loss > current_min_loss_in_total*1.05 :
            print('Saving model with test loss of {:.3f}'.format(test_mean_loss))
            torch.save(bart.state_dict(), weight_path_save+str(epoch_idx)+'_best_model')
# Save at end of epochs
torch.save(bart, weight_path_save+'best_model')
tokenizer.save_pretrained('/kaggle/working/tokenizer')

epoch: 0
Epoch: 0 | avg.loss: 1.590 | lr: 9.89890149147093e-05
Validation
Epoch: 0 | Validation loss 0.7128245970234275
epoch: 1
Epoch: 1 | avg.loss: 0.678 | lr: 9.69688309368581e-05
Validation
Epoch: 1 | Validation loss 0.5472389766946435
epoch: 2
Epoch: 2 | avg.loss: 0.540 | lr: 9.494864695900689e-05
Validation
Epoch: 2 | Validation loss 0.4799954588524997
epoch: 3
Epoch: 3 | avg.loss: 0.439 | lr: 9.292846298115567e-05
Validation
Epoch: 3 | Validation loss 0.4399337135255337
epoch: 4
Epoch: 4 | avg.loss: 0.374 | lr: 9.090827900330446e-05
Validation
Epoch: 4 | Validation loss 0.41688641253858805
epoch: 5
Epoch: 5 | avg.loss: 0.324 | lr: 8.888809502545325e-05
Validation
Epoch: 5 | Validation loss 0.4109106510877609
epoch: 6
Epoch: 6 | avg.loss: 0.284 | lr: 8.686791104760203e-05
Validation
Epoch: 6 | Validation loss 0.4119171276688576
epoch: 7
Epoch: 7 | avg.loss: 0.249 | lr: 8.484772706975082e-05
Validation
Epoch: 7 | Validation loss 0.4134677750989795
epoch: 8
Epoch: 8 | avg.loss: 0.2

In [ ]:
plt.plot(master_train_losses,color = 'green')
plt.plot(master_val_losses,color = 'blue')
plt.title("Traing/validation loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

# Manual Testing

In [ ]:
# test_sentence = test_dataset[0]['translation']['en']
# print('Raw input text:', test_sentence)

# input_ids = encode_input_str(
#     text = test_sentence,
#     target_lang = 'ja',
#     tokenizer = tokenizer,
#     seq_len = model.config.max_length,
#     lang_token_map = LANG_TOKEN_MAPPING)
# input_ids = input_ids.unsqueeze(0).cuda()

# print('Truncated input text:', tokenizer.convert_tokens_to_string(
#     tokenizer.convert_ids_to_tokens(input_ids[0])))

In [ ]:
# output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=3)
# # print(output_tokens)
# for token_set in output_tokens:
#     print(tokenizer.decode(token_set, skip_special_tokens=True))

In [ ]:
# #@title Slick Blue Translate
# input_text = 'A surfboarder ran into a shark' #@param {type:"string"}
# output_language = 'ja' #@param ["en", "ja", "zh"]

# input_ids = encode_input_str(
#     text = input_text,
#     target_lang = output_language,
#     tokenizer = tokenizer,
#     seq_len = model.config.max_length,
#     lang_token_map = LANG_TOKEN_MAPPING)
# input_ids = input_ids.unsqueeze(0).cuda()

# output_tokens = model.generate(input_ids, num_beams=20, length_penalty=0.2)
# print(input_text + '  ->  ' + \
#       tokenizer.decode(output_tokens[0], skip_special_tokens=True))